In [73]:
import numpy as np
import pandas as pd

In [74]:
X = pd.read_csv('train_values.csv')
y = pd.read_csv('train_labels.csv')

In [75]:
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder()
lenc.fit(y.status_group)
labeled_values = lenc.transform(y.status_group)
y['labels'] = labeled_values

In [76]:
year_df = pd.DataFrame({'year':X.construction_year,'label':y.status_group})
year_total = year_df.groupby('year').size()
year_label = year_df.groupby(['year','label']).size()
year_norm = year_label.div(year_total, level = 'year')

In [83]:
import datetime
def preproc(X,top_funders,top_installers,*y):
    #feature engineering
    for year in set(X.construction_year):
#         print(year)
        if 0 in [value for value in year_norm[year].keys().values]:
            X.loc[X.construction_year == year,'prob_f'] = year_norm[year][0]
        else: 
            X.loc[X.construction_year == year,'prob_f'] = 0
        if 1 in [value for value in year_norm[year].keys().values]:
            X.loc[X.construction_year == year,'prob_fnr'] = year_norm[year][1]
        else: 
            X.loc[X.construction_year == year,'prob_fnr'] = 0
        if 2 in [value for value in year_norm[year].keys().values]:
            X.loc[X.construction_year == year,'prob_nf'] =year_norm[year][2]
        else: 
            X.loc[X.construction_year == year,'prob_nf'] = 0   
    
    pred_date = "2014-01-01"
    X['date_recorded_offset_days']=[(datetime.datetime.strptime(pred_date, '%Y-%m-%d') - datetime.datetime.strptime(date, '%Y-%m-%d')).days for date in X.date_recorded]
    X['month recorded'] = [str(datetime.datetime.strptime('2011-02-01', '%Y-%m-%d').month) for date in X.date_recorded]
   
    X.extraction_type = ['other' if extype == 'other - mkulima/shinyanga' else extype for extype in X.extraction_type] #not in test data
#     X.construction_year = X.construction_year.replace(0,1987)
    X.construction_year = X.construction_year.astype('object')
    X.funder = ['Other' if val not in top_funders else val for val in X.funder]
    X.installer = ['Other' if val not in top_installers else val for val in X.installer]
    
    cols_with_nulls = X.isnull().any()[X.isnull().any()].keys()
    for col in cols_with_nulls:
        X.loc[X[col].isnull(), col] = 'Other'
    
    #encoding categorical features
    drop_list = ['construction_year','date_recorded', 'wpt_name', 'subvillage','ward','recorded_by','scheme_name','extraction_type_class','extraction_type_group','payment','quantity']
    categorical_features = list(X.select_dtypes(include = ['object']).drop(drop_list,axis = 1).columns.values)
    X_cat = X[categorical_features]
    X_cat_ohe = pd.get_dummies(X_cat)
    
    preproc_X = X_cat_ohe.join(X.select_dtypes(include = ['int64','float64']).drop(['id','num_private','latitude','longitude'],axis = 1))
        
    return preproc_X

In [78]:
y.status_group.value_counts()/len(y)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [79]:
# Failed experiments

#     OVERFITTING (with mean):
#     tsh_z_f_locs = [(a and b) for a,b in zip(X.amount_tsh==0, y.status_group=='functional')]
#     tsh_z_nf_locs = [(a and b) for a,b in zip(X.amount_tsh==0, y.status_group=='non functional')]
#     tsh_z_fnr_locs = [(a and b) for a,b in zip(X.amount_tsh==0, y.status_group=='functional needs repairs')]    
    
#     X.loc[tsh_z_f_locs, 'amount_tsh'] = tsh_vals['tsh_f']
#     X.loc[tsh_z_nf_locs, 'amount_tsh'] = tsh_vals['tsh_nf']
#     X.loc[tsh_z_fnr_locs, 'amount_tsh'] = tsh_vals['tsh_fnr']

# #     Testing - This didn't work

#     tsh_z_f_locs = [(a and b) for a,b in zip(X.amount_tsh==0, y.status_group=='functional')]
#     tsh_z_nf_locs = [(a and b) for a,b in zip(X.amount_tsh==0, y.status_group=='non functional')]
#     tsh_z_fnr_locs = [(a and b) for a,b in zip(X.amount_tsh==0, y.status_group=='functional needs repairs')]    
    
#     X.loc[tsh_z_f_locs, 'amount_tsh'] = tsh_vals['tsh_f']
#     X.loc[tsh_z_nf_locs, 'amount_tsh'] = tsh_vals['tsh_nf']
#     X.loc[tsh_z_fnr_locs, 'amount_tsh'] = tsh_vals['tsh_fnr']

# Removing FNR:
# y2 = y.copy()
# y2.loc[y2.status_group=='functional needs repair','status_group'] = 'non functional'
# lenc.fit(y2.status_group)
# labeled_values = lenc.transform(y2.status_group)
# y2['labels'] = labeled_values

In [80]:
top_funders = X.funder.value_counts().keys()[0:30]
top_installers = X.installer.value_counts().keys()[0:30]

# tsh_nz_f_locs = [(a and b) for a,b in zip(X.amount_tsh!=0, y.status_group=='functional')]
# tsh_nz_nf_locs = [(a and b) for a,b in zip(X.amount_tsh!=0, y.status_group=='non functional')]
# tsh_nz_fnr_locs = [(a and b) for a,b in zip(X.amount_tsh!=0, y.status_group=='functional needs repairs')]

# tsh_vals = {
#     'tsh_f':X.loc[tsh_nz_f_locs].amount_tsh.median(),
#     'tsh_nf':X.loc[tsh_nz_nf_locs].amount_tsh.median(),
#     'tsh_fnr':X.loc[tsh_nz_fnr_locs].amount_tsh.median()
# }
    

In [84]:
X_pp = preproc(X,top_funders,top_installers)

#Remove features that don't appear in the test data
X_pp = X_pp.drop(['scheme_management_None'],axis=1) 

In [85]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=150, min_samples_leaf=2 ,max_features='sqrt', random_state=314)
from sklearn.metrics import log_loss, accuracy_score

In [86]:
from sklearn.cross_validation import StratifiedKFold 
eval_size = 0.2
n_folds = round(1./eval_size)
kf = StratifiedKFold(y['labels'], n_folds = n_folds, random_state=314)
# total = 0
acc_total = 0
for train_indices, valid_indices in kf:
    X_train, y_train = X_pp.iloc[train_indices], y['labels'].iloc[train_indices]
    X_valid, y_valid = X_pp.iloc[valid_indices], y['labels'].iloc[valid_indices]
    clf.fit(X_train,y_train)
    clf_pred = clf.predict(X_valid)
    acc_score = accuracy_score(y_valid, clf_pred)
    acc_total += acc_score
    print('acc: '+str(acc_score))
#     clf_probs = clf.predict_proba(X_valid)
#     sig_ll_score = log_loss(y_valid, clf_probs)
#     print('log_loss = '+str(sig_score))
#     total += sig_score
# avg_score = total/n_folds
# print('mean log_loss = '+str(avg_score))
avg_acc = acc_total/n_folds
print('mean acc = '+str(avg_acc))

acc: 0.805403585557
acc: 0.800858513593
acc: 0.80303030303
acc: 0.802693602694
acc: 0.803249705338
mean acc = 0.803047142042


In [13]:
X_pp.shape

(59400, 361)

In [303]:
# from sklearn.cross_validation import StratifiedKFold 
# eval_size = 0.1
# n_folds = round(1./eval_size)
# kf = StratifiedKFold(y['labels'], n_folds = n_folds, random_state=314)
# # total = 0
# acc_total = 0
# for train_indices, valid_indices in kf:
#     X_train, y_train = X_pp.iloc[train_indices], y['labels'].iloc[train_indices]
#     X_valid, y_valid = X_pp.iloc[valid_indices], y['labels'].iloc[valid_indices]
#     clf.fit(X_train,y_train)
#     clf_pred = clf.predict(X_valid)
#     acc_score = accuracy_score(y_valid, clf_pred)
#     acc_total += acc_score
#     print('acc: '+str(acc_score))
# #     clf_probs = clf.predict_proba(X_valid)
# #     sig_ll_score = log_loss(y_valid, clf_probs)
# #     print('log_loss = '+str(sig_score))
# #     total += sig_score
# # avg_score = total/n_folds
# # print('mean log_loss = '+str(avg_score))
# avg_acc = acc_total/n_folds
# print('mean acc = '+str(avg_acc))

acc: 0.817539134826
acc: 0.816024238344
acc: 0.814172698199
acc: 0.809123043259
acc: 0.822053872054
acc: 0.810942760943
acc: 0.81228956229
acc: 0.808385250042
acc: 0.805859572319
acc: 0.81980464803
mean acc = 0.81361947803


In [233]:
#y_train.value_counts()/len(y_train)

In [234]:
#y_valid.value_counts()/len(y_valid)

In [235]:
#clf

In [60]:
from sklearn.grid_search import GridSearchCV
param_test1 = {
 'class_weight':[None,"balanced"],
}
gsearch1 = GridSearchCV(clf,param_test1,scoring='accuracy')
gsearch1.fit(X_pp,y['labels'])


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=314, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'class_weight': [None, 'balanced']},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [61]:
gsearch1.grid_scores_

[mean: 0.80589, std: 0.00180, params: {'class_weight': None},
 mean: 0.77981, std: 0.00230, params: {'class_weight': 'balanced'}]

In [62]:
bestclf = gsearch1.best_estimator_
bestclf

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=314, verbose=0, warm_start=False)

In [21]:
# bestclf.fit(X_pp,y['labels'])
clf.fit(X_pp,y2['labels'])
X_test = pd.read_csv('test_values.csv')
X_test_pp = preproc(X_test,top_funders,top_installers)

In [15]:
X_test_pp.shape

(14850, 361)

In [22]:
test_predictions = clf.predict(X_test_pp)
text_labels = {0:'functional', 1:'functional needs repair', 2:'non functional'}
report = pd.DataFrame({'id':X_test.id,'pred':test_predictions})
report['status_group'] = report.pred.map(text_labels)
report = report.drop('pred',axis = 1)
report.to_csv('rep_RFC_no_fnr2.csv', index = False)

In [312]:
tsh_nz_f_locs = [(a and b) for a,b in zip(X_test.amount_tsh!=0, report.status_group=='functional')]
tsh_nz_nf_locs = [(a and b) for a,b in zip(X_test.amount_tsh!=0, report.status_group=='non functional')]
tsh_nz_fnr_locs = [(a and b) for a,b in zip(X_test.amount_tsh!=0, report.status_group=='functional needs repairs')]

tsh_vals = {
    'tsh_f':X_test.loc[tsh_nz_f_locs].amount_tsh.mean(),
    'tsh_nf':X_test.loc[tsh_nz_nf_locs].amount_tsh.mean(),
    'tsh_fnr':X_test.loc[tsh_nz_fnr_locs].amount_tsh.mean()
}

X_test_pp = preproc(X_test,report,top_funders,top_installers,tsh_vals)

TypeError: tuple indices must be integers or slices, not str

In [25]:
X_pp2.shape

(59400, 341)

In [27]:
X_test_pp2.shape

(14850, 341)

In [32]:
report.status_group.value_counts()

functional                 10223
non functional              4598
functional needs repair       29
Name: status_group, dtype: int64